<a href="https://colab.research.google.com/github/chxbim/FP_ML_TomatoLeafDisease/blob/main/FP_ML_TomatoLeafDisease_Pertanian.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
from tqdm import tqdm #visualisasi progress bar
from sklearn.preprocessing import LabelEncoder #encoder label bt data disk npy
#tf keras install + mobileNet
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from sklearn.preprocessing import LabelEncoder
import pickle #obj serialization

In [2]:
#============================================check qty files train/val
for cls in os.listdir(val_dir):
    cls_path = os.path.join(val_dir, cls)
    n_files = len(os.listdir(cls_path))
    print(f"{cls}: {n_files} files")

for cls in os.listdir(train_dir):
    cls_path = os.path.join(train_dir, cls)
    n_files = len(os.listdir(cls_path))
    print(f"{cls}: {n_files} files")

Tomato___Spider_mites Two-spotted_spider_mite: 100 files
Tomato___Septoria_leaf_spot: 100 files
Tomato___Tomato_Yellow_Leaf_Curl_Virus: 100 files
Tomato___Late_blight: 100 files
Tomato___Early_blight: 100 files
Tomato___healthy: 100 files
Tomato___Tomato_mosaic_virus: 100 files
Tomato___Target_Spot: 100 files
Tomato___Leaf_Mold: 100 files
Tomato___Bacterial_spot: 100 files
Tomato___Early_blight: 1000 files
Tomato___Target_Spot: 1000 files
Tomato___healthy: 1000 files
Tomato___Tomato_mosaic_virus: 1000 files
Tomato___Leaf_Mold: 1000 files
Tomato___Tomato_Yellow_Leaf_Curl_Virus: 1000 files
Tomato___Septoria_leaf_spot: 1000 files
Tomato___Bacterial_spot: 1000 files
Tomato___Spider_mites Two-spotted_spider_mite: 1000 files
Tomato___Late_blight: 1000 files


In [3]:
#============================================std img ratio
IMG_SIZE = (224, 224)

#============================================load pretrained model (extractor)
base_model = MobileNetV2(weights='imagenet', include_top=False, pooling='avg')

def extract_feature(img_path):
    img = load_img(img_path, target_size=IMG_SIZE)
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    feat = base_model.predict(img, verbose=0)
    return feat.flatten()

def extract_dataset_features(root_dir):
    features = []
    labels = []
    classes = sorted(os.listdir(root_dir))

    for cls in classes:
        cls_path = os.path.join(root_dir, cls)
        if not os.path.isdir(cls_path):
            continue
        print(f"Processing class: {cls}")

        for img_name in tqdm(os.listdir(cls_path)):
            img_path = os.path.join(cls_path, img_name)
            feat = extract_feature(img_path)
            features.append(feat)
            labels.append(cls)

    return np.array(features), np.array(labels)


/tmp/ipython-input-4209194113.py:4: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights='imagenet', include_top=False, pooling='avg')


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [4]:
x_train, y_train = extract_dataset_features(train_dir)
x_val, y_val = extract_dataset_features(val_dir)

#============================================endocode classes ke int arr
encoder = LabelEncoder()
y_train_enc = encoder.fit_transform(y_train)
y_val_enc   = encoder.transform(y_val)

#============================================sv as npy disk files
np.save("x_train.npy", x_train)
np.save("x_val.npy", x_val)
np.save("y_train.npy", y_train_enc)
np.save("y_val.npy", y_val_enc)

#============================================sving pkl label encoder
with open("label_encoder.pkl", "wb") as f:
    pickle.dump(encoder, f)

print("Saved all features!")


Processing class: Tomato___Bacterial_spot


100%|██████████| 1000/1000 [04:08<00:00,  4.03it/s]


Processing class: Tomato___Early_blight


100%|██████████| 1000/1000 [03:18<00:00,  5.04it/s]


Processing class: Tomato___Late_blight


...

Processing class: Tomato___Tomato_mosaic_virus


100%|██████████| 100/100 [00:19<00:00,  5.08it/s]


Processing class: Tomato___healthy


100%|██████████| 100/100 [00:18<00:00,  5.35it/s]


Saved all features!


In [5]:
#============================================sving pls n npy ke drive (bubble atas biar one run)
np.save("/content/drive/MyDrive/dataset FP ML/x_train.npy", x_train)
np.save("/content/drive/MyDrive/dataset FP ML/x_val.npy", x_val)
np.save("/content/drive/MyDrive/dataset FP ML/y_train.npy", y_train_enc)
np.save("/content/drive/MyDrive/dataset FP ML/y_val.npy", y_val_enc)
with open("/content/drive/MyDrive/dataset FP ML/label_encoder.pkl", "wb") as f:
    pickle.dump(encoder, f)

In [6]:
#============================================baseline: DecisionTree
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import numpy as np

#============================================path to drive
base_path = "/content/drive/MyDrive/dataset FP ML/"

#============================================reload data
x_train = np.load(base_path + "x_train.npy")
x_val   = np.load(base_path + "x_val.npy")
y_train = np.load(base_path + "y_train.npy")
y_val   = np.load(base_path + "y_val.npy")

#============================================baseline model
model = DecisionTreeClassifier(max_depth=None, random_state=42)
model.fit(x_train, y_train)

y_pred = model.predict(x_val)

#============================================eval predict
print("Accuracy:", accuracy_score(y_val, y_pred))
print("\nKlasifikasi:\n", classification_report(y_val, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_val, y_pred))


Accuracy: 0.509

Klasifikasi:
               precision    recall  f1-score   support

           0       0.66      0.55      0.60       100
           1       0.36      0.39      0.37       100
           2       0.52      0.62      0.56       100
           3       0.44      0.46      0.45       100
           4       0.41      0.41      0.41       100
           5       0.47      0.47      0.47       100
           6       0.43      0.41      0.42       100
           7       0.76      0.64      0.70       100
           8       0.48      0.59      0.53       100
           9       0.69      0.55      0.61       100

    accuracy                           0.51      1000
   macro avg       0.52      0.51      0.51      1000
weighted avg       0.52      0.51      0.51      1000


Confusion Matrix:
 [[55  8  4  4  9  5  6  4  4  1]
 [ 4 39 13 13 10  6  4  3  6  2]
 [ 2 13 62  3  7  4  1  3  2  3]
 [ 4  8  8 46  7  5  3  2 16  1]
 [ 6 12 16  7 41  4  6  1  7  0]
 [ 0  6  1  7  3 47 12  3